In [1]:
import pyspark
from operator import add, sub
from time import sleep
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.streaming.kafka import KafkaUtils
import json
from pyspark import SparkContext, SparkConf


In [2]:
# conf = SparkConf().set("spark.jars", "/path-to-jar/spark-streaming-kafka-0-8-assembly_2.11-2.2.1.jar")

# sc = SparkContext( conf=conf)
# spark = SparkSession.builder.appName('my_awes').master("spark://bphani.local:7077").config("spark.jars", "/Users/phaneendra/Downloads/spark-sql-kafka-0-10_2.11-2.4.4.jar,/Users/phaneendra/Downloads/kafka-clients-2.4.0.jar").getOrCreate()
spark = SparkSession.builder.appName('my_awes').master("local").config("spark.jars", "/Users/phaneendra/Downloads/spark-sql-kafka-0-10_2.11-2.4.4.jar,/Users/phaneendra/Downloads/kafka-clients-2.4.0.jar").getOrCreate()



In [6]:
# val mySchema = StructType(Array(
#  StructField("id", IntegerType),
#  StructField("name", StringType),
#  StructField("year", IntegerType),
#  StructField("rating", DoubleType),
#  StructField("duration", IntegerType)
# ))
movie_schema = StructType().add("id", "integer")\
.add("name", "string").add("year", "integer").add("rating", "double").add("duration", "integer")



df = spark.read.schema(movie_schema).csv('/Users/phaneendra/Downloads/moviedata.csv', inferSchema=True, header=True)

In [20]:
from pyspark.sql import functions as F

newDf2 = df.withColumn("value", F.to_json(F.struct([F.when(F.col(x)!="  ",df[x]).otherwise(None).alias(x) for x in df.columns])))
newDf2.show(truncate=False)

+---+---------------------------+----+------+--------+--------------------------------------+
|id |name                       |year|rating|duration|value                                 |
+---+---------------------------+----+------+--------+--------------------------------------+
|2  |The Mummy                  |1932|3.5   |4388    |{"name":"The Mummy"}                  |
|3  |Orphans of the Storm       |1921|3.2   |9062    |{"name":"Orphans of the Storm"}       |
|4  |The Object of Beauty       |1991|2.8   |6150    |{"name":"The Object of Beauty"}       |
|5  |Night Tide                 |1963|2.8   |5126    |{"name":"Night Tide"}                 |
|6  |One Magic Christmas        |1985|3.8   |5333    |{"name":"One Magic Christmas"}        |
|7  |Muriel's Wedding           |1994|3.5   |6323    |{"name":"Muriel's Wedding"}           |
|8  |Mother's Boys              |1994|3.4   |5733    |{"name":"Mother's Boys"}              |
|9  |Nosferatu: Original Version|1929|3.5   |5651    |{"name

In [24]:
newDf2.columns

['id', 'name', 'year', 'rating', 'duration', 'value']

In [22]:
#Worked
newDf2.selectExpr("CAST(year AS INTEGER)", "CAST(value AS STRING)") \
  .write \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("topic", "test_kafka_spark") \
  .save()



In [25]:
#Worked
newDf2.selectExpr("CAST(year AS STRING) AS key", "to_json(struct(*)) AS value") \
  .write \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("topic", "test_kafka_spark_2") \
  .save()

In [26]:
#Try for SalesRecords

sdf = spark.read.csv('/Users/phaneendra/Downloads/100000SalesRecords.csv', inferSchema=True, header=True)
sdf.columns

['Region',
 'Country',
 'Item Type',
 'Sales Channel',
 'Order Priority',
 'Order Date',
 'Order ID',
 'Ship Date',
 'Units Sold',
 'Unit Price',
 'Unit Cost',
 'Total Revenue',
 'Total Cost',
 'Total Profit']

In [28]:
sdf.selectExpr("CAST(Region AS STRING) AS key", "to_json(struct(*)) AS value") \
  .write \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("topic", "test_kafka_spark_3") \
  .save()

In [3]:
dft1 = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "localhost:9092").option("subscribe", "topic-one").option("startingOffsets", "latest").load()



In [4]:
dft1.writeStream.format("kafka").option("topic", "topic-one").option("kafka.bootstrap.servers", "localhost:9092").option("checkpointLocation","/tmp/").start()




In [3]:
prob_schema = StructType().add("CUST_ID", "string").add("OUTBUFFER__1", "string")
# Read CSV files from set path
dfCSV = spark.readStream.option("sep", ",").option("header", "true").schema(prob_schema).csv("/Users/phaneendra/Downloads/prob*.csv")


In [4]:
dfCSV.createOrReplaceTempView("prob")
total = spark.sql("select count(CUST_ID) from prob")


In [5]:
dfCSV \
.write  \
.format("kafka") \
.option("kafka.batch.size", 5000) \
.option("kafka.bootstrap.servers", "localhost:9092") \
.option("kafka.request.timeout.ms", 120000) \
.option("topic", "raw") \
.option("checkpointLocation", "/mnt/telemetry/cp.txt") \
.save()

AnalysisException: "'write' can not be called on streaming Dataset/DataFrame;"

In [ ]:
query = total.writeStream.outputMode("complete").format("console").start()
query.awaitTermination()

In [ ]:
query.close()

In [9]:
dfCSV.selectExpr("CAST(CUST_ID AS STRING) AS key", "to_json(struct(*)) AS value").
  writeStream
  .format("kafka")
  .option("topic", "topic-one")
  .option("kafka.bootstrap.servers", "localhost:9092")
  .start()

SyntaxError: invalid syntax (<ipython-input-9-090b7f70e295>, line 1)